In [1]:
from scraper.datamodel import DbApi

In [3]:
db = DbApi('../data/backup2.db')

In [4]:
db.conn.execute('select count(*) from comments').fetchall()

[(1459660,)]

In [6]:
db.conn.execute('select count(*) from submissions').fetchall()

[(222211,)]

In [7]:
db.conn.execute('select count(*) from sentences').fetchall()

[(5816712,)]

In [15]:
db.conn.execute('select src_is_subm, count(*) from sentences group by 1').fetchall()

[(0, 2908670), (1, 2908042)]

In [14]:
db.conn.execute('select text, count(*) from sentences where src_is_subm=1 group by 1 order by 2 desc limit 50').fetchall()

[("I don't know what to do.", 3356),
 ("I don't know.", 1857),
 ('I want to die.', 1671),
 ('I hate myself.', 1086),
 ("I don't know what to do anymore.", 1061),
 ('I need help.', 1035),
 ('I just want to die.', 1009),
 ('What do I do?', 862),
 ('Please help me.', 794),
 ('I have no friends.', 646),
 ("I can't do this anymore.", 644),
 ('Thanks for reading.', 638),
 ('What should I do?', 616),
 ("I'm so tired.", 567),
 ("I just don't know what to do.", 513),
 ("What's the point?", 505),
 ('I need help', 421),
 ("I can't take it anymore.", 415),
 ('I give up.', 391),
 ('I want to kill myself.', 389),
 ('What can I do?', 385),
 ("I don't want to die.", 382),
 ("I don't know what to do", 374),
 ('I hate my life.', 370),
 ('I hate it.', 366),
 ("I just don't know what to do anymore.", 320),
 ("I can't do it anymore.", 315),
 ('I have nothing.', 314),
 ("I don't know why.", 304),
 ("I don't want to live anymore.", 304),
 ('I want to die', 304),
 ('I feel so alone.', 289),
 ('Thank you for r

In [21]:
qry_nobots = """
with bots as (
select author, count(*)
from comments 
group by 1
having count(*) >=20
union all
select 'SeriouslyLucifer', -1
union all
select 'peadditer', -1
)
select text, count(*) 
from sentences s
join comments c
  on s.src_id = c.id
  and src_is_subm=0 
left join bots b
  on c.author = b.author
where b.author is null
and c.author <> '[deleted]'
group by 1 order by 2 desc limit 50
"""
db.conn.execute(qry_nobots).fetchall()

[('Thank you so much!', 334),
 ('I love it.', 198),
 ('Username checks out', 177),
 ("I don't know.", 174),
 ('I like it.', 163),
 ('What do you mean?', 160),
 ('Thank you :)', 152),
 ('Thanks so much!', 133),
 ('I love it!', 121),
 ('Not at all.', 116),
 ('Thanks for sharing.', 112),
 ('Best of luck!', 111),
 ('Thank you so much.', 111),
 ('Thank you very much!', 107),
 ('What are you talking about?', 106),
 ("I don't think so.", 102),
 ('Thanks for the reply.', 102),
 ("I don't get it.", 100),
 ('Thanks for the info.', 100),
 ('Thanks for the info!', 95),
 ('I agree with you.', 94),
 ('Thanks for sharing!', 92),
 ('Happy cake day!', 91),
 ('Thanks a lot!', 91),
 ('( ͡° ͜ʖ ͡°)', 89),
 ('That makes sense.', 88),
 ('Hope this helps!', 87),
 ('Username checks out.', 87),
 ('Thank you for your submission!', 84),
 ('Good to know.', 80),
 ('Hope this helps.', 80),
 ('Thanks for the advice.', 76),
 ('Keep it up!', 75),
 ('I get it.', 73),
 ('I feel you.', 69),
 ('Yes it is.', 69),
 ('Thanks 

In [17]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = ? or s.text = ? 
order by 1, 3, 4 desc, 2
"""
bad_sents = ['___\n\n[**Here are our subreddit rules.', "**Unfortunately, we've had to remove your post."]

db.conn.execute(qry_lowfreq_bots, bad_sents).fetchall()

[('SeriouslyLucifer',
  'FortniteCompetitive',
  "**Unfortunately, we've had to remove your post.",
  190,
  "**Unfortunately, we've had to remove your post.** \n\n___\n\n###Posts Should Be High Quality And Encourage Discussion\n\nWe want /r/FortniteCompetitive to be the best source of discussion for the game online - and to do that we have to ensure that quality content isn’t drowned out by low effort or low-quality submissions.  Content must be related to Fortnite. If your post contains a suggestion for the game, it must contain Fortnite elements.\n\n**High Quality Guidelines:**\n\n- Posts about similar games are not allowed.\n\n- Every post should be a space for meaningful discussion. While this doesn't mean every post needs to be a discursive essay, posts that don't offer room for meaningful discussion will be removed.\n\n- Screenshots are removed at moderation discretion if they do not contain sufficient supporting text to encourage a discussion.\n\n- Videos/images taken with a ph

In [19]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = 'Thank you for your submission!'
order by 1, 3, 4 desc, 2
"""

db.conn.execute(qry_lowfreq_bots).fetchall()

[('peadditer',
  'CoDCompetitive',
  'Thank you for your submission!',
  87,
  "Thank you for your submission! Unfortunately, your submission has been removed for the following reason(s):\n\nPlease use the 'Daily Discussion Thread' stickied at the top of the subreddit for your question/topic. Thanks!\n\n\n\nIf you have any questions regarding the ruleset of /r/CoDCompetitive, please refer to the [FAQ](/r/CoDCompetitive/wiki/index) or [message the moderators](/message/compose?to=%2Fr%2FCoDCompetitive). Thank you!")]